In [6]:
import numpy as np
from PIL import Image
import math
import random

from ray import *
from hittable import *
from camera import *

def degrees_to_radians(doubledegrees):
    return degrees * pi / 180.0;
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm
def color(array):
    c = np.multiply(255.0, array)
    return c

In [7]:
# return the color of a ray
# form: np.array(255, 255, 255)
# world: hittable_list of all objects
def ray_color(r, world):
    rec = hit_record()
    hit_something, rec = world.hit(r, 0, math.inf, rec)
    if (hit_something):
#         print(rec.normal)
        return 0.5 * color(rec.normal + np.array([1, 1, 1]))
    
#     No Object hit (Background)
    unit_direction = normalize(r.direction)
    t = 0.5 * (unit_direction[1] + 1)
    return (1.0 - t) * color(np.array([1, 1, 1])) + t * color(np.array([0.5, 0.7, 1.0]))

In [8]:
# main

# Image
image_width = 160
image_height = 120
aspect_ratio = image_width / image_height

# PIL preprocessing to use PIL coordinates
img = Image.new(mode = "RGB", size = (image_width, image_height), color = 0)
img_array = np.array(img)
img_array = np.swapaxes(img_array, 1, 0)

# Camera
viewport_height = 2
viewport_width = aspect_ratio * 2
focal_length = 1

# World object list
world = hittable_list()
world.add(sphere(np.array([0, 0, -1]), 0.5))
world.add(sphere(np.array([0, -100.5, -1]), 100))




origin = np.array([0, 0, 0])
horizontal = np.array([viewport_width, 0, 0])
vertical = np.array([0, viewport_height, 0])
lower_left_corner = origin - horizontal / 2 - vertical / 2 - np.array([0, 0, focal_length])

# Render
print("image width: ", image_width," image height: ", image_height)
for j in reversed(range(image_height)):
    if j % 50 == 0:
        print(j, "Scanlines remaining")
    for i in range(image_width):
        u = i / (image_width - 1)
        v = j / (image_height - 1)
        r = Ray(origin, lower_left_corner + u * horizontal + v * vertical - origin)
        img_array[i][j] = ray_color(r, world)

print("Done")

image width:  160  image height:  120
100 Scanlines remaining
50 Scanlines remaining
0 Scanlines remaining
Done


In [9]:
final_img_array = np.swapaxes(img_array, 1, 0)
final_img_array = np.flip(final_img_array, 0)
new_pic = Image.fromarray(final_img_array.astype('uint8'), 'RGB')
new_pic.show()